
# Incorporating OLI Calculations with WaterTAP

#### Contact: Paul Vecchiarelli (paul.vecchiarelli@nrel.gov)

This tutorial will demonstrate basic usage of OLI Cloud calls using our custom API tools.

## Rationale

 - Simulations for realistic water sources are mathematically complex: 
 > $ Interactions \ge Cations * Anions$
 - OLI improves WaterTAP approximations and offloads computational resources

## Required OLI API Inputs


 - State variables (solute concentrations, temperature, pressure), which can beextracted from a state block
 
 - Login credentials
 
 - A chemistry (*.dbs) file
     - establishes state variables, phases, etc. to be considered in flash calls

In [ ]:
# used to specify state/convert variables
from pyomo.environ import units as pyunits

# used to build survey lists
from numpy import linspace

# used to execute OLI Cloud functions
from watertap.tools.oli_api.flash import Flash
from watertap.tools.oli_api.credentials import CredentialManager
from watertap.tools.oli_api.client import OLIApi

# 1. Specify State Variables.

- This data is used to construct inputs to OLI Cloud

In [ ]:
source_water = {
    "temperature": 298.15,
    "pressure": 101325,
    "components": {
        "Cl_-": 870,
        "Na_+": 739,
        "SO4_2-": 1011,
        "Mg_2+": 90,
        "Ca_2+": 258,
        "K_+": 9,
        "HCO3_-": 385,
        "SiO2": 30,
    },
    "units": {
        "temperature": pyunits.K,
        "pressure": pyunits.Pa,
        "components": pyunits.mg / pyunits.L,
    },
}

# 2. Initialize Flash Instance.

 - We will run most of our methods with this class

In [ ]:
f = Flash()

# 3. Get Survey Parameters.

 - In this example, we will generate a temperature sweep survey

In [ ]:
survey_conditions = {
    "Temperature": linspace(273, 373, 10),
    "SO4_2-": linspace(0, 1e3, 10),
    "Ca_2+": linspace(0, 1e3, 10),
}

survey = f.build_survey(
    survey_conditions,
    get_oli_names=True,
)

# 4. Login to OLI Cloud.

- The following code demonstrates an OLI Cloud login:

In [ ]:
try:
    credential_manager = CredentialManager(
        username="",
        password="",
        root_url="",
        auth_url="",
        access_keys=[],
    )
except (OSError, ConnectionError) as e:
    print(e)
    credential_manager = None

# 5. Create *.dbs File and 6. Get Output

In [ ]:
if credential_manager:
    with OLIApi(credential_manager) as oliapi:
        # create a new DBS file
        dbs_file_id = oliapi.get_dbs_file_id(
            chemistry_source=source_water["components"],
            phases=["liquid1", "solid"],
        )
        
        # create water analysis inputs
        water_analysis_input = f.build_flash_calculation_input(
            "wateranalysis",
            state_vars,
        )
    
        # create water analysis case
        water_analysis_base_case = f.run_flash(
            "wateranalysis",
            oliapi,
            dbs_file_id,
            water_analysis_input,
            file_name="water_analysis_singlepoint"
        )
        
        # create apparent composition from water analysis output
        water_analysis_apparent_composition = f.build_flash_calculation_input(
            "isothermal",
            state_vars,
            water_analysis_base_case[0],
        )
        
        # run isothermal flash for the survey parameters
        isothermal_survey_result = f.run_flash(
            "isothermal",
            oliapi,
            dbs_file_id,
            water_analysis_apparent_composition,
            survey,
            "isothermal_composition_survey",
        )

# 7. Extract Filtered Output

 - OLI's output is robust, so WaterTAP enables printing selected results:

In [ ]:
# extract properties from raw oli output
properties = [
    "prescalingTendencies",
    "entropy",
    "gibbsFreeEnergy",
    "selfDiffusivities",
    "molecularConcentration",
    "kValuesMBased",
]
if credential_manager:
    extracted_properties = f.extract_properties(
        isothermal_survey_result,
        properties,
        filter_zero=True,
        file_name="properties",
    )